# MedCompare

## Requirements

Attention: before running, switch to gpu execution

In [2]:
!pip install datasets
!pip install transformers
!pip install huggingface_hub
!pip install tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from huggingface_hub import notebook_login

In [4]:
notebook_login()

## 1. Data Processing


In [5]:
import re
import json
import requests
from transformers import AutoTokenizer, pipeline, AutoModel, pipeline
import torch
from datasets import load_dataset
from functools import lru_cache

Load Data

In [6]:
dataset = load_dataset("MattBastar/Medicine_Details")
data = dataset['train']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Medicine_Details.csv:   0%|          | 0.00/4.36M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11825 [00:00<?, ? examples/s]

We get the ontology mapping via open source platform BioPortal bioontology. For this u need a file (api-key.txt) that contains an api-key for BioPortal. For this create an account5 at https://bioportal.bioontology.org/ and get your own key.

In [7]:
# BASE_URL = "http://data.bioontology.org"
# file = open("api-key.txt", "r")
# API_KEY = file.read().strip()
# file.close()

# headers = {
#     "Authorization": f"apikey token={API_KEY}"
# }

In [8]:
# Function to clean text

# def clean_text(text):
#     text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove special characters
#     text = re.sub(r"\b\d+\b", "", text)  # Remove standalone numbers
#     text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
#     return text

def clean_text(text):
    # Step 1: Remove special characters except for alphanumeric and spaces
    text = text.lower()
    text = re.sub(r"[^a-zA-Z\s]", "", text)

    # Step 2: Remove standalone numbers
    text = re.sub(r"\b\d+\b", "", text)

    # Step 3: Remove specific unwanted terms
    unwanted_terms = ["mg", "treatment", "Treatment", "MG", "mg", "ML", "ml", "of", "mgml"]
    for term in unwanted_terms:
        text = re.sub(rf"\b{term}\b", "", text, flags=re.IGNORECASE)

    # Step 4: Remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()

    return text



In [9]:
# # Function to look up ontology mappings from BioPortal API

# def get_bioportal_mapping(term):

#     params = {
#         "q": term,
#         "require_exact_match": "false" # false or true  deoending on exact matching
#     }
#     response = requests.get(f"{BASE_URL}/search", headers=headers, params=params)

#     if response.status_code != 200:
#         return {term: "unknown"}  # Default to "unknown" if the API call fails

#     data = response.json()

#     # Filter relevant mappings based on ontology prefixes else words without medical context get mapped as well
#     relevant_prefixes = [
#         "http://purl.bioontology.org/ontology",  # BioPortal's main prefix
#         "http://www.co-ode.org/ontologies/galen",  # GALEN ontology
#         "http://ncicb.nci.nih.gov"  # NCI Thesaurus
#     ]

#     for result in data.get("collection", []):
#         label = result.get("prefLabel")
#         ontology_id = result.get("@id")

#         if label and ontology_id and any(ontology_id.startswith(prefix) for prefix in relevant_prefixes):
#             return {label.lower(): ontology_id}

#     # Default
#     return {term: "unknown"}

### Processing User Input


In [278]:
pipe = pipeline("token-classification", model="Clinical-AI-Apollo/Medical-NER", aggregation_strategy="simple")

def preprocess_user_input_with_ner(user_input):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Step 1: Use the NER pipeline to extract medical entities
    ner_results = pipe(user_input)

    # Step 2: Group all entities by their types
    grouped_entities = {}
    for entity in ner_results:
        entity_group = entity["entity_group"]
        if entity_group not in grouped_entities:
            grouped_entities[entity_group] = []
        grouped_entities[entity_group].append(entity["word"])

    # Step 3: Collect all identified words
    all_words = []
    for group, words in grouped_entities.items():
        all_words.extend(words)

    # Step 4: Combine all words into a single string
    combined_text = " ".join(all_words)
    print(f"Combined Text: {combined_text}")

    # Step 5: Generate a single embedding for the combined text
    inputs = tokenizer(combined_text, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        combined_embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()  # CLS token embedding

    # Step 6: Return results
    return {
        "cleaned_input": user_input,
        "ner_results": ner_results,
        "grouped_entities": grouped_entities,
        "all_words": all_words,
        "combined_text": combined_text,
        "combined_embedding": combined_embedding  # Single combined embedding
    }

def preprocess_user_input_with_ner_tfidf(user_input):
    # Step 1: Use the NER pipeline to extract medical entities
    ner_results = pipe(user_input)

    # Step 2: Group all entities by their types
    grouped_entities = {}
    for entity in ner_results:
        entity_group = entity["entity_group"]
        if entity_group not in grouped_entities:
            grouped_entities[entity_group] = []
        grouped_entities[entity_group].append(entity["word"])

    # Step 3: Collect all identified words
    all_words = []
    for group, words in grouped_entities.items():
        all_words.extend(words)

    # Step 4: Combine all words into a single string
    combined_text = " ".join(all_words)
    print(f"Combined Text: {combined_text}")

    tfidf_embeddings = vectorizer.transform([combined_text])

    print(tfidf_embeddings.shape)
    # Step 6: Return results
    return {
        "cleaned_input": user_input,
        "ner_results": ner_results,
        "grouped_entities": grouped_entities,
        "all_words": all_words,
        "combined_text": combined_text,
        "combined_embedding": tfidf_embeddings  # Feature names for interpretation
    }


Device set to use cpu


###Preprocessing for User Input aswell as Dataset

In [11]:
def preprocess_with_ner(text_input):
    ner_results = pipe(text_input)
    all_words = [entity["word"] for entity in ner_results]
    combined_text = " ".join(all_words)
    return combined_text

In [12]:
from tqdm import tqdm
def combine_text(record):
    composition = clean_text(record['Composition'])
    uses = clean_text(record['Uses'])
    return f"{composition}{uses}"


processed_texts = [preprocess_with_ner(combine_text(record)) for record in tqdm(data, desc="Processing records")]



Processing records: 100%|██████████| 11825/11825 [1:30:28<00:00,  2.18it/s]


In [13]:
import csv

output_file = "processed_texts.csv"

# Save to a CSV file
with open(output_file, "w", encoding="utf-8", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Processed Text"])  # Header row
    for text in processed_texts:
        writer.writerow([text])

print(f"Processed texts saved to {output_file}")
from google.colab import files

# Download the CSV file
files.download(output_file)


Processed texts saved to processed_texts.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Generate and store Dataset embeddings with Clinical-BERT

In [134]:
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

def generate_embeddings(processed_text, model, tokenizer):
  print("Executing with Cuda GPU: " + str(torch.cuda.is_available()))
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = model.to(device)
  embeddings = []
  for text in tqdm(processed_text, desc="Generating embeddings"):
      inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(device)
      with torch.no_grad():
          outputs = model(**inputs)
      embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
      embeddings.append(embedding)

  # Convert embeddings to a NumPy array
  entity_embedding = np.array(embeddings)
  return entity_embedding


data_embeddings_bert = generate_embeddings(processed_texts, model, tokenizer)


Executing with Cuda GPU: False


Generating embeddings: 100%|██████████| 11825/11825 [28:18<00:00,  6.96it/s]


In [135]:

# embeddings_file = "data_embeddings_bert.npy"
# np.save(embeddings_file, data_embeddings_bert)

# print(f"BERT Embeddings saved to {embeddings_file}")


BERT Embeddings saved to data_embeddings_bert.npy


### Generate and store Dataset embeddings with TFIDF

In [280]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=1000, stop_words=None)

def generate_tfidf_embeddings(processed_texts):
    processed_texts = [text for text in processed_texts if text.strip()]

    tfidf_embeddings = vectorizer.fit_transform(processed_texts)

    return tfidf_embeddings


data_embeddings_tfidf = generate_tfidf_embeddings(processed_texts)


In [262]:
embeddings_file = "data_embeddings_tfidf.npy"
np.save(embeddings_file, data_embeddings_tfidf)

print(f"TFIDF Embeddings saved to {embeddings_file}")


## 2. Similarity Ranking

Prepare Data Embeddings and Query Embeddings for Ranking

TODO: maybe add normalization before embedding

In [239]:
from sklearn.metrics.pairwise import cosine_similarity

function for user query preprocessing

In [274]:
def prepare_query_for_similarity_ranking(query, embedding_type = 'bert'):

  query = clean_text(query)

  if(embedding_type == 'tfidf'):
    preprocessed_query = preprocess_user_input_with_ner_tfidf(query)
  else:
    preprocessed_query = preprocess_user_input_with_ner(query)

  print("NER Results:", preprocessed_query["ner_results"])
  print(f"NER Query for Embedding: {preprocessed_query['combined_text']}")

  query_embedding = preprocessed_query["combined_embedding"]

  # print(group_embedding.shape)
  # print(embedding_type)
  if(embedding_type == 'bert'):
    query_embedding = np.array(query_embedding).reshape(1, -1)

  return query_embedding


function for ranking best Matches and Eliminate Duplicates, because necessary only Substances and Dataset provides multiple solutions with different dosages


In [241]:
def similarity_ranking(similarities, data, top_k = 5):

  medicine_names = [record['Medicine Name'] for record in data]
  compositions = [record['Composition'] for record in data]
  uses= [record['Uses'] for record in data]
  side_effects = [record['Side_effects'] for record in data]
  ratings = [record['Average Review %'] for record in data]


  ranked_indices = np.argsort(similarities[0])[::-1]  # Sort indices by similarity in descending order

  unique_compositions = set()
  final_results = []  # To store the final unique top-k results

  for idx in ranked_indices:
      medicine_name = medicine_names[idx]
      composition = compositions[idx]
      use = uses[idx]
      side_effect = side_effects[idx]
      average_review = ratings[idx]

      # Check for uniqueness based on composition
      if clean_text(composition) not in unique_compositions:
          unique_compositions.add(clean_text(composition)) # Mark composition as seen
          final_results.append({
              "Medicine": medicine_name,
              "Composition": composition,
              "Use": use,
              "Side Effects": side_effect,
              "Similarity": similarities[0][idx]
          })

      if len(final_results) == top_k:
          break
  return final_results

function for printing top results

In [242]:
def print_results(similarity_results):
  print("\nTop matching medications with side effects:")
  for result in similarity_results:
      print(f"Medicine: {result['Medicine']},\nComposition: {result['Composition']}, Use: {result['Use']}, \nSide Effects: {result['Side Effects']},\n Similarity: {result['Similarity']:.4f}\n")

  print("\n" + "=" * 50 + "\n")

###Calculating Cosine Similarities with Sklearn
and combine all together

In [243]:
def similarity(query, data_embedding, ground_data, embedding_type = 'bert'):
    query_embedding = prepare_query_for_similarity_ranking(query, embedding_type)
    #cosine similarity sklearn
    similarities = cosine_similarity(query_embedding, data_embedding)
    similarity_results = similarity_ranking(similarities, ground_data)
    print_results(similarity_results)


###Testing of Similarity Ranking

In [281]:
query = "acute migraine"
similarity(query, data_embeddings_tfidf, data, embedding_type = 'tfidf')
similarity(query, data_embeddings_bert, data)

Combined Text: acute migraine
(1, 1000)
NER Results: [{'entity_group': 'DETAILED_DESCRIPTION', 'score': 0.23664123, 'word': 'acute', 'start': 0, 'end': 5}, {'entity_group': 'DISEASE_DISORDER', 'score': 0.13441671, 'word': 'migraine', 'start': 5, 'end': 14}]
NER Query for Embedding: acute migraine
(1, 1000)

Top matching medications with side effects:
Medicine: Zavedos 5mg Injection,
Composition: Idarubicin (5mg), Use:  Blood cancer (Acute lymphocytic leukemia), 
Side Effects: Vomiting Nausea Infection Hair loss Fever Increased liver enzymes Peripheral neuropathy tingling and numbness of feet and hand Stomatitis Inflammation of the mouth Bleeding Myelosuppression Seizure,
 Similarity: 0.6446

Medicine: Pinoin 0.025% Cream,
Composition: Tretinoin (0.025%), Use: Treatment of Acne Acute promyelocytic leukaemia, 
Side Effects: Erythema skin redness Skin irritation Dry skin Dry lips Pale red skin Increased sensitivity to light,
 Similarity: 0.4453

Medicine: Tretwin Cream,
Composition: Treti

## 3. Evaluation of Clinical-BERT & TFIDF results
